# A Flight Through a Terra Wormhole
> There's a new bridge in town

- toc:true
- branch: master
- badges: true
- comments: false
- author: Scott Simpson
- categories: [Terra, Wormhole]
- hide: false  


In [1]:
#hide
#Imports & settings
!pip install plotly --upgrade
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline
%load_ext google.colab.data_table
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))


     |████████████████████████████████| 25.3 MB 67 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


dplyr.summarise.inform,[RTYPES.NILSXP]


In [10]:
#hide
%%R

#Grab wormhole bridge query from Flipside
df_wh = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/355ce3fd-4e1a-4043-89e7-eddbdbff3aa7/data/latest', simplifyDataFrame = TRUE)

#fix the column names
names(df_wh)<-tolower(names(df_wh))

#Change the date to date format
df_wh$block_timestamp <- parse_datetime(df_wh$block_timestamp)


#Grab eth shuttle bridge query from Flipside
df_sh_eth = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/97d5a217-63e2-465b-a2a0-b7013678678b/data/latest', simplifyDataFrame = TRUE)
#Grab bsc receive shuttle bridge query from Flipside
df_sh_bsc_r = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/17027191-c807-4709-b8a2-66b12e7e0d74/data/latest', simplifyDataFrame = TRUE)
#Grab bsc send shuttle bridge query from Flipside
df_sh_bsc_s = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/10fdf58b-4e5c-4f9a-a492-550062101b40/data/latest', simplifyDataFrame = TRUE)
#Grab harmony  shuttle bridge query from Flipside
df_sh_harm = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/0252f148-8572-45a7-8f27-a344f1a69b51/data/latest', simplifyDataFrame = TRUE)


#pull into one table
df_sh <- df_sh_eth %>%
  bind_rows(df_sh_bsc_r) %>%
  bind_rows(df_sh_bsc_s) %>%
  bind_rows(df_sh_harm)

#tidy up
rm(df_sh_bsc_r)
rm(df_sh_bsc_s)
rm(df_sh_eth)
rm(df_sh_harm)

#fix the column names
names(df_sh)<-tolower(names(df_sh))
df_sh <- df_sh %>% rename(bridge_chain_id = shuttle_chain_id)
df_wh <- df_wh %>% rename(bridge_chain_id = wormhole_chain_id)

#Change the date to date format
df_sh$block_timestamp <- parse_datetime(df_sh$block_timestamp)

#join chain ids
chains <- tibble(bridge_chain_id = c(0,1,2,3,4,5,6),
                 bridge_chain = c('Unknown','Solana','Ethereum','Terra','BSC','Polygon','Harmony'))
df_sh <- df_sh %>% left_join(chains, by = "bridge_chain_id")
df_wh <- df_wh %>% left_join(chains, by = "bridge_chain_id")

#merge the tables
df_sh$bridge <- 'Shuttle'
df_wh$bridge <- 'Wormhole'
df <- df_sh %>% bind_rows(df_wh)

#create a date field 
df$date <- floor_date(df$block_timestamp, unit = 'day')

# clip by date to remove the last part date
df <- df %>% filter(date < '2021-11-17')

#Grab the token labels
labels <- read_csv("https://github.com/scottincrypto/analytics/raw/master/data/wormhole_bridge_assets.csv", show_col_types = FALSE)

#join the token lables
df <- df %>%
  left_join(labels, by = "denom")


#transactions per day
tx_by_day <- df %>%
  group_by(bridge, date) %>%
  summarise(tx_count = n(),
            user_count = n_distinct(user)
  )

#destinations by day
tx_by_day_chain <- df %>%
  group_by(bridge, date, bridge_chain) %>%
  summarise(tx_count = n(),
            user_count = n_distinct(user)
  )

#coins by day
tx_by_day_coin <- df %>%
  group_by(bridge, date, group2, group2_order) %>%
  summarise(tx_count = n(),
            user_count = n_distinct(user)
  ) 

#transactions by day as a percentage of total
tx_by_day_percent <- tx_by_day %>% select(-tx_count) %>% pivot_wider(names_from = bridge, values_from = user_count, values_fill = 0) %>% 
    mutate('Shuttle Bridge' = Shuttle / (Shuttle + Wormhole) * 100, 'Wormhole Bridge' = Wormhole / (Shuttle + Wormhole) * 100) %>%
    select(-Shuttle, -Wormhole) %>%  pivot_longer(cols=-date, names_to = "bridge", values_to = "percentage") %>% 
    arrange(date, desc(bridge)) 



# The Wormhole Bridge

The Terra Network, up until recently, has had limited options for blockchain-blockchain transfer of assets.  The official Terra Bridge, known as the Shuttle Bridge, has served Terra with bridges to Ethereum, Binance Smart Chain (BSC) and the Harmony blockchains.  The Shuttle Bridge is a centralised conduit between chains, with liquidity held by multisig wallets.  Terraform Labs put forward the plan to migrate to a decentralised alternative - the Wormhole Network.

The Wormhole Network is a cross chain network which solves the problem of inter-chain communication between blockchains which have different consensus mechanisms.  It operates a set of Guardian nodes which perform attestations on the consensus operations of the attached chains.  This simple model removes the need to run a consensus protocol on Wormhole - it is simply a network of oracles reporting on what happens on the attached chains.  Thus Wormhole is able to manage communications between networks with very different consensus models - in particular, Solana.

The Columbus-5 upgrade to Terra provided the infrastructure for Wormhole to interoperate with the network. This went live in late September 2021 on the Terra Network.  In October 2021, the Wormhole V2 network went live, providing a bridge between Terra, Ethereum, BSC and Solana, with Polygon added a few weeks later.  We will examine the uptake of this exciting new product.

[![Wormhole Network](https://miro.medium.com/max/700/0*z3bTqkYsG3hs3X1Q)](https://medium.com/terra-money/terra-goes-live-on-wormhole-v2-12df49d446d2)

# Transaction Volume

The data used in this analysis is based on transactions to & from the Terra network using either the Shuttle Bridges (one each for Ethereum, BSC and Harmony) or the Wormhole Bridge (a single contract endpoint for all destinations).  The chart below shows the number of transactions across the two bridge networks since the Columbus-5 upgrade on Terra on September 30 2021. 

Here we see a large increase in transactions across the Shuttle bridge in the weeks following the Columbus-5 upgrade.  This settled down to a steady level of around 3000 per day prior to the Wormhole launch.  Wormhole transactions quickly jumped up to 200-300 per day and remained steady there.  The additional transactions don't appear to have had a dramatic impact on the overall number of bridge transactions.

In [11]:
#hide_input
#Plot the proportion of transactions
df_rel_debt = %R tx_by_day %>% arrange(desc(bridge))
fig = px.area(df_rel_debt
              , x="date"
              , y="tx_count"
              , color="bridge"
              , template="simple_white", width=800, height=800/1.618
              , title= "Bridge Transactions by Bridge")
fig.update_yaxes(title_text='Count of all Bridge Transactions')
fig.update_xaxes(title_text=None)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.90,
    xanchor="right",
    x=0.99
    
))
fig.update_layout(legend_title_text=None)

fig.show()

If we examine the above data, but plot each bridge as a percentage of the total, we see that the Wormhole bridge traffic is around 6-9% of the total bridge traffic to & from Terra.  The prior observations of this being a steady state hold - there doesn't appear to be any dramatic ramp-up in traffic.

In [12]:
#hide_input
#Plot the percentages of transactions
df_rel_debt = %R tx_by_day_percent
fig = px.area(df_rel_debt
              , x="date"
              , y="percentage"
              , color="bridge"
              , template="simple_white", width=800, height=800/1.618
              , title= "Percentage of Bridge Transactions by Bridge")
fig.update_yaxes(title_text='% of all Bridge Transactions')
fig.update_xaxes(title_text=None)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.90,
    xanchor="right",
    x=0.99
    
))
fig.update_layout(legend_title_text=None)

fig.show()

# User Adoption Rate

Using the same dataset, we can examine the number of unique wallets on each day which interact with the bridges.  Whilst people can have multiple wallets, this is the best proxy we have for user adoption.  The chart below shows the number of wallets interacting with the bridge on each day.  Wormhole appears to be attracting 100-200 users per day, with it appearing to be steadily rising.  This doesn't appear to be having a discernable impact on the number of users on the Shuttle bridge.  It's possible these may be new bridge users - it's difficult to tell from this dataset but some further data will support this hypothesis.

In [13]:
#hide_input
# User Counts
df_p = %R tx_by_day %>% select(-tx_count) %>% pivot_wider(names_from = bridge, values_from = user_count, values_fill = 0) %>% arrange(date)

fig = make_subplots(rows=2, cols=1, subplot_titles=("Shuttle Bridge Wallets", "Wormhole Bridge Wallets"))
fig.append_trace(go.Scatter(x=df_p["date"], y=df_p["Shuttle"], name="Shuttle Wallets Count"), row=1, col=1)
fig.append_trace(go.Scatter(x=df_p["date"], y=df_p["Wormhole"], name="Wormhole Wallets Count"), row=2, col=1)
fig.update_layout(width=800, height=800/1.618, title_text="Bridge Wallet Count")
fig.update_layout(template="simple_white", showlegend=False)
fig.update_yaxes(title_text='Wallet Count', row=1, col=1)
fig.update_yaxes(title_text='Wallet Count', row=2, col=1)
fig.show()

# Which Tokens are Transferred on the Bridges?

The Shuttle bridge supports sending & receiving of wrapped Terra assets to the remote bridges.  Wormhole extends this functionality and supports sending & receiving of many more assets from other chains.  These exist as wrapped versions of the original token on the Terra chain.

It should come as no surprise that the majority of bridge traffic is LUNA and UST - the two keystone assets on the Terra network.  UST has the potential to be a major cross-chain decentralisd US dollar stablecoin, and LUNA is the volatile asset which gives users exposure to the success of the Terra Network.  The following charts show the types of tokens which are sent & received on the two bridges, broken into groups as follows:
* UST
* LUNA
* Other Terra Assets - Terra native assets like ANC, MIR, and KRT and Mirror assets like mAAPL and mTSLA
* Non-Terra Assets - assets from other chains wrapped by Wormhole - whETH, whUSDT, whDOGE etc

The first chart below shows the number of transactions on the Shuttle Bridge split by the token type from the list above.  UST accounts for most of the traffic, demonstrating its utility as a cross-chain stablecoin.  A smaller number of transactions are LUNA (remembering that 1 LUNA ~ 50 UST) and an even smaller amount are the other Terra assets.


In [14]:
#hide_input
#Tokens Sent/Received via Wormhole
df_p = %R tx_by_day_coin %>% filter(bridge == "Shuttle") %>% arrange(group2_order)
fig = px.bar(df_p
             , x = "date"
             , y = "tx_count"
             , color = 'group2'
             , labels=dict(date="Date", tx_count="Transactions", group2='Token Type')
             , title= "Transactions on Shuttle Bridge by Token Type"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='Transaction Count')
fig.update_xaxes(title_text=None)
fig.show()

The next chart shows the number of transactions by token type over the Wormhole bridge.  The results are similar - most transactions are for UST.  We can see the non-Terra assets with a small but consistent usage - there is some demand for new assets on Terra.

In [15]:
#hide_input
#Tokens Sent/Received via Wormhole
df_p = %R tx_by_day_coin %>% filter(bridge == "Wormhole") %>% arrange(group2_order)
fig = px.bar(df_p
             , x = "date"
             , y = "tx_count"
             , color = 'group2'
             , labels=dict(date="Date", tx_count="Transactions", group2='Token Type')
             , title= "Transactions on Wormhole Bridge by Token Type"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    title_text=None
))
fig.update_yaxes(title_text='Transaction Count')
fig.update_xaxes(title_text=None)
fig.show()

# What are the Destinations?

The two bridges have a different set of linked chains, so we expect to see a difference in the data when we look at the source & destination chains.  The Shuttle bridge is only linked to Ethereum, Binance Smart Chain (BSC) and Harmony, whereas Wormole supports Ethereum, BSC, Polygon and Solana.  It is expected that Wormhole will expand to support more chains in the future.  

The Shuttle bridge shows most of the transactions are to or from the BSC chain.  This behaviour can potentially be explained by the high fees associated with bridging to Ethereum in comparison to BSC.  Whilst bridge fees themselves are modest, the gas costs for getting on or off at the Ethereum end of the bridge are very high.

In [16]:
#hide_input
#Destination Chain Sent via Shuttle (received not possible)
df_p = %R tx_by_day_chain %>% filter(bridge == "Shuttle") %>% filter(bridge_chain != "Unknown") %>% arrange(bridge_chain)
fig = px.bar(df_p
             , x = "date"
             , y = "tx_count"
             , color = 'bridge_chain'
             , labels=dict(date="Date", tx_count="Transactions", bridge_chain='Chain')
             , title= "All Transactions on Shuttle Bridge by Source/Destination Chain"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='Transaction Count')
fig.update_xaxes(title_text=None)
fig.show()

The chart below shows all of the transactions sent on via the Wormhole bridge with their destination chains.  Note that this data does not contain transactions received via Wormhole - the source chain is not visible in the interchain message which comes from the bridge to the Terra network.  The data below is surprising - it doesn't mirror the Shuttle bridge data.  We see that the overwhelming majority of transactions sent via Wormhole are to the Solana network.  Users of the Shuttle bridge haven't migrated their bridging activities across to Wormhole - they have a new use case instead.  It appears that the Wormhole bridge has immediately serviced an unmet need - transferring assets between Terra and Solana.

A further insight can be drawn from these data - why haven't users migrated across to the Wormhole bridge from the Shuttle bridge?  The fees on the Shuttle bridge are 0.1% of the transfer value (minimum fee 1UST) plus gas fees at either end, whereas the Wormhole fees are currently sub-cent plus gas.  Wormhole is the cheaper option.  It's possible that the Shuttle bridge users simply don't yet know about the newer, cheaper bridge.  It may take further marketing or changing of the Shuttle bridge UI to move users to Wormhole.

In [17]:
#hide_input
#Destination Chain Sent via Wormhole (received not possible)
df_p = %R tx_by_day_chain %>% filter(bridge == "Wormhole") %>% filter(bridge_chain != "Unknown") %>% arrange(bridge_chain)
fig = px.bar(df_p
             , x = "date"
             , y = "tx_count"
             , color = 'bridge_chain'
             , labels=dict(date="Date", tx_count="Transactions", bridge_chain='Destination Chain')
             , title= "Send Transactions on Wormhole Bridge by Destination Chain"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    title_text=None
))
fig.update_yaxes(title_text='Transaction Count')
fig.update_xaxes(title_text=None)
fig.show()

# Conclusions

The Wormhole Network is a new cross-chain bridge network, linking blockchains with different consensus mechanisms in a decentralised manner.  This network is the successor to the existing Shuttle bridge on the Terra Network.  We have seen that the Wormhole Network has seen modest growth in transaction volume since launch, but still only a few hundred per day.  It doesn't appear to have impacted the usage of the Shuttle bridge, which hosts an order of magnitude more transactions.  User adoption on Wormhole is slowly rising, but interestingly hasn't appeared to impact use of the Shuttle bridge.  UST is the dominant asset transferred across both bridges, and we have seen a new class of non-Terra assets appear on Terra via the Wormhole Bridge.  Finally, we saw that most of the Shuttle bridge traffic was to or from Binance Smart Chain.  We expected similar on Wormhole (assume that users would simply migrate), but instead saw that most of the traffic was to Solana, servicing a previously unmet need.  We drew the conclusion that users simply don't yet know about Wormhole, as it is a cheaper option for inter-chain transfers at the moment.

All data was sourced from the curated on-chain data tables at [Flipside Crypto](https://flipsidecrypto.com/)